In [3]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import joblib
import warnings

# Ignorer les warnings
warnings.filterwarnings('ignore')

# Fonction pour le prétraitement des données
def preprocess_data(df):
    df.dropna(inplace=True)  # Supprimer les lignes avec des valeurs manquantes
    df.drop_duplicates(inplace=True)  # Supprimer les doublons

    # Encodage des variables catégorielles
    label_encoders = {}
    for column in df.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

    return df, label_encoders

# Charger les données
df = pd.read_csv('fraud.csv')

# Prétraitement des données
df, label_encoders = preprocess_data(df)

# Séparer les caractéristiques et la cible
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardiser les caractéristiques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Gestion des déséquilibres de classe
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Optimisation des hyperparamètres avec RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 10, 20, 30]
}
model = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', random_state=42)
random_search.fit(X_train_res, y_train_res)
best_model = random_search.best_estimator_

print("Meilleurs paramètres : ", random_search.best_params_)

# Évaluation du modèle
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    print("Rapport de Classification :\n", classification_report(y_test, y_pred))
    print("Matrice de Confusion :\n", confusion_matrix(y_test, y_pred))

    # Courbe ROC et AUC
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    roc_auc = roc_auc_score(y_test, y_prob)

    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, marker='.', color='blue')
    plt.title('Courbe ROC')
    plt.xlabel('Taux de Faux Positifs')
    plt.ylabel('Taux de Vrais Positifs')
    plt.grid(True)
    plt.show()

    print(f"AUC : {roc_auc:.4f}")

evaluate_model(best_model, X_test, y_test)

# Validation Croisée
cv_scores = cross_val_score(best_model, X, y, cv=5)
print(f"Scores de Validation Croisée : {cv_scores}")
print(f"Score Moyen : {cv_scores.mean():.4f}")

# Sauvegarder le modèle
joblib.dump(best_model, 'fraud_detection_model.pkl')
print("Modèle sauvegardé avec succès.")


KeyboardInterrupt: 